# Statistical Learning Assignment2

In [16]:
library(MASS)
library(ISLR)
library(class)
library(boot)
library(dplyr)
Default
#cor(Default)

default,student,balance,income
No,No,729.5265,44361.625
No,Yes,817.1804,12106.135
No,No,1073.5492,31767.139
No,No,529.2506,35704.494
No,No,785.6559,38463.496
No,Yes,919.5885,7491.559
No,No,825.5133,24905.227
No,Yes,808.6675,17600.451
No,No,1161.0579,37468.529
No,No,0.0000,29275.268


In [2]:
set.seed(123)
sample_num <-sample(1:nrow(Default), size= 0.8 * nrow(Default), replace=F)

train <- Default[sample_num, ]
test <- Default[-sample_num, ]

In [3]:
mtrx_and_stats<- function(pred,actual){ # the input= prediction and actual data for test set
    #creating confusion matrix
    confusion_matrix<-table(pred,actual)
    print(confusion_matrix)
    #calculating precision, recall, and f-score from the matrix
    precision <- confusion_matrix[2,2] / sum(confusion_matrix[2,])
    recall <- confusion_matrix[2,2] / sum(confusion_matrix[,2])
    F.score <- 2*precision*recall/(precision+recall)
    stats<-c(precision,recall,F.score)
    return(stats)
}

In [4]:
# building model with training data
glm.fit <- glm(default~income+balance+student,
               data=train, family='binomial')
# Predicting with test data
glm.probs<- predict(glm.fit, newdata=test, type='response')
# As glm.probs is not the response 'yes' and 'no',
#I just turned it into the response prediction by creating glm.pred data
glm.pred<- rep('No', nrow(test))
glm.pred[glm.probs>0.5]='Yes'

#Applying the function that I made above
glm_stats<-mtrx_and_stats(glm.pred, test$default)
glm_stats

     actual
pred    No  Yes
  No  1928   46
  Yes    6   20


[1] 0.7692308 0.3030303 0.4347826

In [5]:
summary(glm.fit)


Call:
glm(formula = default ~ income + balance + student, family = "binomial", 
    data = train)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.1526  -0.1404  -0.0558  -0.0199   3.7422  

Coefficients:
              Estimate Std. Error z value Pr(>|z|)    
(Intercept) -1.097e+01  5.487e-01 -19.995   <2e-16 ***
income       4.449e-06  9.095e-06   0.489   0.6248    
balance      5.772e-03  2.605e-04  22.162   <2e-16 ***
studentYes  -6.344e-01  2.621e-01  -2.421   0.0155 *  
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 2340.6  on 7999  degrees of freedom
Residual deviance: 1252.3  on 7996  degrees of freedom
AIC: 1260.3

Number of Fisher Scoring iterations: 8


In [6]:
# building model with training data
lda.fit <- lda(default~income+balance+student,
               data=train)
# Predicting with test data
lda.pred <- predict(lda.fit, newdata=test)
# lda.class is the predicted classification(yes/no)
lda.class=lda.pred$class

#Applying the function that I made above
lda_stats<-mtrx_and_stats(lda.class, test$default)
lda_stats

     actual
pred    No  Yes
  No  1929   51
  Yes    5   15


[1] 0.7500000 0.2272727 0.3488372

In [7]:
lda.fit

Call:
lda(default ~ income + balance + student, data = train)

Prior probabilities of groups:
      No      Yes 
0.966625 0.033375 

Group means:
      income   balance studentYes
No  33561.60  804.6836  0.2890211
Yes 32151.98 1750.1822  0.3820225

Coefficients of linear discriminants:
                     LD1
income      4.437213e-06
balance     2.246048e-03
studentYes -1.513689e-01

In [8]:
# building model with training data
qda.fit <- qda(default~income+balance+student, data=train)
# Predicting with test data
qda.class<- predict(qda.fit, newdata=test)$class
#Applying the function
qda_stats<-mtrx_and_stats(qda.class, test$default)
qda_stats

     actual
pred    No  Yes
  No  1928   49
  Yes    6   17


[1] 0.7391304 0.2575758 0.3820225

In [9]:
qda.fit

Call:
qda(default ~ income + balance + student, data = train)

Prior probabilities of groups:
      No      Yes 
0.966625 0.033375 

Group means:
      income   balance studentYes
No  33561.60  804.6836  0.2890211
Yes 32151.98 1750.1822  0.3820225

In [10]:
# As student predictor is categorial(qualatative, not quantatative),
# it should be turned into numbers.
# So I just created copy dataframe of default just for knn.
Default_for_knn<-data.frame(Default)

Default_for_knn$student <- ifelse(Default_for_knn$student=='Yes', 1,0)

# As it turned into numbers, the student column of train and test set should be also changed,
# so I just repeated the process of splitting the changed dataset into train and test
# like following
sample_num_knn <-sample(1:nrow(Default_for_knn), size= 0.8 * nrow(Default_for_knn), replace=F)
train_knn <- Default_for_knn[sample_num_knn, ]
test_knn <- Default_for_knn[-sample_num_knn, ]

# predicting with the model
knn.pred<- knn(train_knn[,2:4], test_knn[,2:4],
               cl=train_knn$default, k=3)

#trying putting other numbers into k, 3 was the optimal which has highest stats result
knn_stats<- mtrx_and_stats(knn.pred,test_knn$default)
knn_stats

     actual
pred    No  Yes
  No  1915   49
  Yes   22   14


[1] 0.3888889 0.2222222 0.2828283

In [11]:
df=data.frame(glm_stats,lda_stats,qda_stats,knn_stats)
dimnames(df)=list(row=c("precision","recall",'f-score'),col=names(df));df
# selecting the model which has the highest score
#on each calculation(precision, recall and f1-score)

precision_first <- names(df)[max.col(replace(df, is.na(df),0),ties.method = "first")][1]
recall_first <- names(df)[max.col(replace(df, is.na(df),0),ties.method = "first")][2]
f1_score_first <- names(df)[max.col(replace(df, is.na(df),0),ties.method = "first")][3]
cat('model with the highest precision:',precision_first,
   '\n model with the highest recall:',recall_first,
   '\n model with the highest f1-score:',f1_score_first)

,glm_stats,lda_stats,qda_stats,knn_stats
precision,0.7692308,0.7500000,0.7391304,0.3888889
recall,0.3030303,0.2272727,0.2575758,0.2222222
f-score,0.4347826,0.3488372,0.3820225,0.2828283


model with the highest precision: glm_stats 
 model with the highest recall: glm_stats 
 model with the highest f1-score: glm_stats

In [12]:
Default

default,student,balance,income
No,No,729.5265,44361.625
No,Yes,817.1804,12106.135
No,No,1073.5492,31767.139
No,No,529.2506,35704.494
No,No,785.6559,38463.496
No,Yes,919.5885,7491.559
No,No,825.5133,24905.227
No,Yes,808.6675,17600.451
No,No,1161.0579,37468.529
No,No,0.0000,29275.268


In [13]:
# 10-fold cross validation
glm.fit_cv <- glm(default~income+balance+student, data=Default, family='binomial')
cv.error.10 = cv.glm(Default,glm.fit_cv,K=10)$delta[1] #set the k as 10, so the number of fold=10
cv.error.10 #the Cv error(average of MSEs)

[1] 0.02136392

In [14]:
set.seed(123)
#sample_num <-sample(1:nrow(Default), size= 0.8 * nrow(Default), replace=F)
#train <- Default[sample_num, ]
#test <- Default[-sample_num, ]

glm.fit <- glm(default~income+balance+student,
               data=train, family='binomial') #the old-best model
glm.probs <- predict(glm.fit, newdata=test, type='response')
test$default<- ifelse(test$default=='Yes',1,0)
MSE_old_best<-mean((test$default-glm.probs)^2)
test$default<- ifelse(test$default==1,'Yes','No')

ifelse(cv.error.10 > MSE_old_best,
       'Cv error of 10-fold cross validation has higher error than MSE of the old best model',
       'Cv error of 10-fold cross validation has lower error than MSE of the old best model')

[1] "Cv error of 10-fold cross validation has higher error than MSE of the old best model"

In [15]:
#Extra Researching

tenfold<-0
old_best<-0

for (i in 1:100){
    sample_num <-sample(1:nrow(Default), size= 0.8 * nrow(Default), replace=F)
    train <- Default[sample_num, ]
    test <- Default[-sample_num, ]
    glm.fit <- glm(default~income+balance+student,
                   data=train, family='binomial') #the old-best model
    glm.probs <- predict(glm.fit, newdata=test, type='response')
    test$default<- ifelse(test$default=='Yes',1,0)
    MSE_old_best<-mean((test$default-glm.probs)^2)
    test$default<- ifelse(test$default==1,'Yes','No')
    if (cv.error.10 > MSE_old_best){
        tenfold=tenfold+1
        }else{
        old_best=old_best+1
    }}
tenfold # the number of times that Cv error is higher than MSE of old best model
old_best # the number of times that Cv error is lower than MSE of old best model

[1] 51

[1] 49